In [ ]:
import pandas as pd


In [5]:
DATASET_PATH = r"C:\Users\Nikhil Bhandari\Desktop\ML Projects\Fake news classifier\fake-news\train.csv"

In [6]:
df = pd.read_csv(DATASET_PATH)

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


### Checking and droping null values

In [8]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
df = df.dropna()

In [10]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


### Splitting independent and dependent features

In [11]:
X = df.drop('label', axis = 1)
Y = df['label']

In [12]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [13]:
Y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [14]:
import tensorflow as tf
tf.__version__

'2.18.0'

In [ ]:
# pip install --upgrade tensorflow

In [15]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [16]:
vocab_size = 5000

### Onehot representation

In [17]:
messages = X.copy()

In [18]:
messages.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


### Text Preprocessing

In [21]:
import nltk
import re
from nltk.corpus import stopwords

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Nikhil
[nltk_data]     Bhandari\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [23]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [24]:
#Resetting the index
messages = messages.reset_index(drop=True)

In [25]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [26]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [27]:
one_hot_repr = [one_hot(words,vocab_size) for words in corpus]
one_hot_repr

[[781, 2277, 1462, 4951, 620, 1676, 3797, 2950, 1814, 2837],
 [1591, 3670, 1348, 2379, 4824, 2026, 1035],
 [49, 2239, 2046, 1212],
 [950, 1143, 2616, 1648, 2206, 2515],
 [2706, 4824, 2254, 116, 2778, 4527, 4824, 2314, 476, 2747],
 [1261,
  4198,
  2166,
  2466,
  1602,
  789,
  1984,
  3028,
  3296,
  362,
  2079,
  666,
  3855,
  125,
  1035],
 [3503, 3820, 4511, 2706, 799, 3673, 1052, 2856, 2693, 4874, 2247],
 [3559, 178, 3797, 4881, 1214, 1996, 789, 4547, 2693, 4874, 2247],
 [1653, 427, 1777, 4018, 187, 4840, 4643, 445, 789, 262],
 [2471, 1128, 2878, 3962, 3606, 2281, 3279, 25],
 [4816, 4741, 2694, 827, 4173, 2070, 716, 4610, 71, 1706, 3338],
 [1648, 2470, 620, 4840, 789, 1214],
 [3412, 299, 4139, 1926, 4411, 3159, 1520, 4697, 2872],
 [2557, 3889, 2928, 869, 4093, 382, 3335, 2693, 4874, 2247],
 [934, 3975, 828, 2125, 749, 2693, 4874, 2247],
 [3492, 2399, 1463, 2358, 3344, 4221, 3804, 714, 2437, 1060],
 [2847, 4923, 3670],
 [2533, 4096, 347, 4076, 789, 2202, 2321, 1035],
 [3809, 3703

### Now we need to add padding, so that are model can be trained


In [28]:
sent_length = 20
embedded_docs = pad_sequences(one_hot_repr,padding = 'post',maxlen = sent_length)
print(embedded_docs)

[[ 781 2277 1462 ...    0    0    0]
 [1591 3670 1348 ...    0    0    0]
 [  49 2239 2046 ...    0    0    0]
 ...
 [ 801   13 1052 ...    0    0    0]
 [2775 1214 3140 ...    0    0    0]
 [1165 4911 2063 ...    0    0    0]]


In [29]:
embedded_docs[1]

array([1591, 3670, 1348, 2379, 4824, 2026, 1035,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

### Creating LSTM model


In [31]:
embedding_vector_features=40 ##features representation
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Force the model to build by passing a sample input
dummy_input = tf.random.uniform((1, sent_length), minval=0, maxval=vocab_size, dtype=tf.int32)
model(dummy_input)


# Print the summary
print(model.summary())


c:\Users\Nikhil Bhandari\Desktop\ML Projects\Fake news classifier\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (1, 20, 40)            │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (1, 100)               │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (1, 1)                 │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

None


# Force the model to build by passing a sample input
dummy_input = tf.random.uniform((1, sent_length), minval=0, maxval=vocab_size, dtype=tf.int32)
model(dummy_input)

This happend because TensorFlow/Keras lazy-builds models—meaning the actual layer shapes and parameters aren't initialized until the model receives input data for the first time. Hence above we passing an input so that the model gets initialsed

In [32]:
import numpy as np
X_final = np.array(embedded_docs)
Y_final = np.array(Y)


In [33]:
X_final.shape,Y_final.shape

((18285, 20), (18285,))

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test , Y_train, Y_test = train_test_split(X_final,Y_final,test_size = 0.33, random_state = 42)

###Training the model

In [36]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs = 10, batch_size = 64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.7616 - loss: 0.4228 - val_accuracy: 0.9175 - val_loss: 0.1967
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9515 - loss: 0.1278 - val_accuracy: 0.9205 - val_loss: 0.2051
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9756 - loss: 0.0690 - val_accuracy: 0.9203 - val_loss: 0.2471
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9830 - loss: 0.0507 - val_accuracy: 0.9178 - val_loss: 0.2420
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9877 - loss: 0.0384 - val_accuracy: 0.9176 - val_loss: 0.2937
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9913 - loss: 0.0252 - val_accuracy: 0.9122 - val_loss: 0.3580
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9936 - loss: 0.0212 - val_accuracy: 0.9140 - val_loss: 0.3851
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9941 - loss: 0.0199 - val_accu

###Performance metrics and accuracy

In [37]:
y_pred = model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


Convering probabilities to class labels


In [38]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [39]:
from sklearn.metrics import confusion_matrix

Y_test → The actual class labels (ground truth) from the test dataset.

y_pred → The predicted class labels output by the model.

In [40]:
confusion_matrix(Y_test,y_pred)

array([[3189,  230],
       [ 357, 2259]])

In [41]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,y_pred)

0.9027340513670257

In [55]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91      3419
           1       0.90      0.86      0.88      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.89      0.90      6035
weighted avg       0.90      0.90      0.90      6035



In [57]:
model.save("fake_news_classifier.keras")

In [60]:
from tensorflow.keras.models import load_model
model = load_model("fake_news_classifier.keras")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
